In [1]:
# Student name: Jiayan Dong
# PID: A16593051
# Date: 11/3/2022
# [HW5_prob1] Resnet20 Quantization-aware Training (15pts)
# 4 bit

import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "resnet20_quant_4bit"
model = resnet20_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        bit=4,
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        bit=4,
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        bit=4,
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False

In [ ]:
# This cell won't be given, but students will complete the training

lr = 0.12
weight_decay = 1e-4
epochs = 200
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [2]:
# HW 5 Q1 4 bit

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [2]:
PATH = "result/resnet20_quant_4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9161/10000 (92%)



In [3]:
#send an input and grap the value by using prehook like HW3
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu") 

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        # print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
# From discussion question: Question on pre-hooked inputs for Resnet model by Rishav Karki
print("pre-hooked input of BasicBlock 0 conv1 size:", save_output.outputs[0][0].size())
print("pre-hooked input of BasicBlock 0 conv2 size:", save_output.outputs[1][0].size())

pre-hooked input of BasicBlock 0 conv1 size: torch.Size([128, 16, 32, 32])
pre-hooked input of BasicBlock 0 conv2 size: torch.Size([128, 16, 32, 32])


In [4]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-2.0000, -4.0000,  1.0000],
          [ 2.0000,  2.0000, -1.0000],
          [-1.0000, -1.0000, -3.0000]],

         [[-3.0000, -2.0000,  1.0000],
          [ 0.0000,  2.0000, -0.0000],
          [-2.0000, -0.0000,  3.0000]],

         [[-2.0000,  2.0000,  3.0000],
          [-1.0000,  2.0000,  0.0000],
          [-0.0000, -1.0000,  3.0000]],

         ...,

         [[ 0.0000,  1.0000, -2.0000],
          [ 0.0000, -2.0000, -2.0000],
          [-2.0000, -3.0000, -4.0000]],

         [[-4.0000, -2.0000, -1.0000],
          [-4.0000,  2.0000,  2.0000],
          [ 2.0000,  2.0000, -2.0000]],

         [[-1.0000, -3.0000, -2.0000],
          [-1.0000, -6.0000, -2.0000],
          [ 1.0000, -2.0000, -0.0000]]],


        [[[-2.0000, -1.0000, -2.0000],
          [-1.0000, -1.0000, -2.0000],
          [ 0.0000,  0.0000, -0.0000]],

         [[ 1.0000,  2.0000,  1.0000],
          [ 2.0000,  3.0000,  1.0000],
          [-0.0000,  1.0000, -0.0000]],

         [[-0.0000, -0.0000, -0

In [5]:
x_bit = 4
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv1.act_alpha.data.item()
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 4.0000,  7.0000,  7.0000,  ..., 11.0000, 11.0000,  8.0000],
          [ 6.0000,  5.0000,  5.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 6.0000,  5.0000,  5.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 6.0000,  5.0000,  5.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 6.0000,  5.0000,  5.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 8.0000,  7.0000,  7.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 6.0000,  5.0000,  5.0000,  ...,  3.0000,  3.0000,  4.0000],
          [ 4.0000,  3.0000,  3.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.0000,  3.0000,  3.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 4.0000,  3.0000,  3.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.0000,  3.0000,  3.0000,  ...,  2.0000,  2.0000,  0.0000],
          [ 3.0000,  2.0000,  2.0000,  ...,  1.0000,  2.0000,  1.0000]],

         [[ 9.0000,  7.0000,  7.0000,  ...,  8.0000,  8.0000,  7.0000],
          [ 9.0000,  7.0000,  

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int = conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
print(output_recovered)

tensor([[[[-6.7955e+00, -7.7969e+00, -4.1488e+00,  ..., -4.3205e+01,
           -4.3491e+01, -4.1131e+01],
          [-3.1474e+00,  1.9313e+00,  3.2189e+00,  ..., -1.8598e+00,
           -6.4378e-01, -1.0730e+00],
          [-3.1474e+00,  1.5737e+00,  6.4378e+00,  ...,  4.4350e+00,
            5.6510e+00,  4.4350e+00],
          ...,
          [-3.1474e+00,  1.2876e+00,  6.3663e+00,  ..., -1.4592e+01,
           -1.5236e+01, -1.2876e+01],
          [-3.1474e+00, -2.1460e-01,  9.7283e+00,  ..., -9.6568e+00,
           -5.0072e+00, -4.2204e+00],
          [-1.0157e+01, -9.0130e+00,  6.2232e+00,  ...,  3.5050e+00,
            4.7211e+00,  1.0730e+01]],

         [[-3.0043e+00,  5.7225e-01, -6.5094e+00,  ..., -4.4350e+00,
           -4.8641e+00, -5.0072e+00],
          [-4.8641e+00,  3.5050e+00, -8.5123e+00,  ..., -1.7883e+01,
           -1.7668e+01, -2.0315e+01],
          [-4.8641e+00,  2.2890e+00, -1.0157e+01,  ..., -1.6524e+01,
           -1.5952e+01, -1.6667e+01],
          ...,
     

In [7]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q

output_ref = conv_ref(x)
print(output_ref)

tensor([[[[-9.9109e+00, -7.7873e+00,  1.6801e+00,  ..., -5.7274e+01,
           -5.7148e+01, -5.5514e+01],
          [-2.4342e+00,  1.4008e+00,  9.8916e+00,  ...,  3.5265e+00,
            4.0531e+00,  4.2261e+00],
          [-2.4342e+00,  8.6847e-01,  1.1628e+01,  ...,  4.3933e+00,
            5.0204e+00,  4.1230e+00],
          ...,
          [-2.4342e+00,  8.7689e-01,  1.4324e+01,  ..., -1.4866e+01,
           -1.6437e+01, -1.3349e+01],
          [-2.4342e+00, -9.9194e-02,  1.3860e+01,  ..., -9.4060e+00,
           -4.9161e+00, -4.1402e+00],
          [-7.9783e+00, -6.2920e+00,  9.2196e+00,  ...,  2.8603e+00,
            4.8348e+00,  1.1294e+01]],

         [[-4.0816e+00, -5.0769e-01, -7.4869e+00,  ..., -3.9331e+00,
           -3.9982e+00, -4.3924e+00],
          [-5.8585e+00,  3.7767e+00, -8.4273e+00,  ..., -1.5813e+01,
           -1.5864e+01, -1.8643e+01],
          [-5.8585e+00,  2.6268e+00, -1.0348e+01,  ..., -1.4392e+01,
           -1.4352e+01, -1.5495e+01],
          ...,
     

In [8]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-8.6829e+00, -6.5874e+00,  3.4381e+00,  ..., -5.8517e+01,
           -5.8411e+01, -5.6792e+01],
          [-1.4482e+00,  2.6912e+00,  1.1980e+01,  ...,  5.4040e+00,
            5.9397e+00,  6.3451e+00],
          [-1.4482e+00,  2.2051e+00,  1.4133e+01,  ...,  6.0916e+00,
            6.7747e+00,  5.9097e+00],
          ...,
          [-1.4482e+00,  2.1284e+00,  1.6964e+01,  ..., -1.4233e+01,
           -1.5751e+01, -1.1995e+01],
          [-1.4482e+00,  1.0222e+00,  1.6407e+01,  ..., -9.1235e+00,
           -3.8491e+00, -2.8323e+00],
          [-7.9653e+00, -6.5323e+00,  1.0560e+01,  ...,  3.7281e+00,
            6.2175e+00,  1.2831e+01]],

         [[-5.0118e+00, -2.3241e+00, -7.6565e+00,  ..., -6.3027e+00,
           -6.3461e+00, -6.8867e+00],
          [-7.0263e+00,  1.5945e+00, -8.1081e+00,  ..., -1.7894e+01,
           -1.7900e+01, -2.0513e+01],
          [-7.0263e+00,  2.2109e-01, -1.0501e+01,  ..., -1.5732e+01,
           -1.5632e+01, -1.6752e+01],
          ...,
     

In [10]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.5989, device='cuda:0', grad_fn=<MeanBackward0>)


In [11]:
difference.mean()/output_ref.abs().mean()

tensor(0.1118, device='cuda:0', grad_fn=<DivBackward0>)